In [41]:
from one.api import ONE
import one
from utils import download_glm_hmm, save_data_path, val_eid

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import matplotlib.patches as mpatches


In [3]:
# Loading up the cleaned data
data = pd.read_pickle('data/ephys_glm_frame_data.pkl')
data.shape

(29023, 31)

In [4]:
# add a column for the signed contrast (right - left). 
# if nan, treat it as 0
data["signed_contrast"] = data["contrastRight"].fillna(0) - data["contrastLeft"].fillna(0)

In [5]:
data.columns

Index(['stimOnTrigger_times', 'repNum', 'goCueTrigger_times',
       'quiescencePeriod', 'contrastLeft', 'rewardVolume', 'probabilityLeft',
       'firstMovement_times', 'goCue_times', 'contrastRight', 'feedback_times',
       'feedbackType', 'choice', 'stimOn_times', 'response_times', 'session',
       'stimOff_times', 'glm-hmm_5', 'glm-hmm_3', 'glm-hmm_2', 'glm-hmm_4',
       'intervals_0', 'intervals_1', 'intervals_bpod_0', 'intervals_bpod_1',
       'subject', 'included', 'trial_idx', 'frame_preds', 'frame_start_index',
       'frame_end_index', 'signed_contrast'],
      dtype='object')

In [6]:
def count_max_prob_classes(frame_predictions):
    # Find the index of the max probability in each row
    max_indices = np.argmax(frame_predictions, axis=1)
    
    # Count occurrences of each index
    counts = np.bincount(max_indices, minlength=4)
    
    # Map counts to class names
    class_names = ["still", "move", "wheel_turn", "groom"]
    count_dict = {class_names[i]: counts[i] for i in range(4)}
    
    return count_dict

In [7]:
adf = pd.DataFrame()

for sess_id in val_eid:
    # Get the data for the session
    sess_data = data[data['session'] == sess_id]
    
    # for every trial of this session, get the frame predictions
    for i in range(len(sess_data)):
        # Get the data for this trial
        trial_data = sess_data.iloc[i]
        
        # Get the frame predictions
        frame_predictions = trial_data['frame_preds']

        # Count the number of occurrences of each class
        count_dict = count_max_prob_classes(frame_predictions)

        # get the 3 state state model
        state_model = trial_data['glm-hmm_4']

        # rest of the columns
        rest = trial_data[[
            'session', 'trial_idx', 'signed_contrast',
            'stimOnTrigger_times', 'repNum', 'goCueTrigger_times',
            'quiescencePeriod', 'contrastLeft', 'stimOn_times', 'feedback_times',
            'probabilityLeft', 'response_times', 'rewardVolume', 'choice',
            'goCue_times', 'firstMovement_times', 'contrastRight', 'feedbackType',
            'stimOff_times', 'intervals_0', 'intervals_1', 'intervals_bpod_0',
            'intervals_bpod_1', 'subject', 'included',
            'frame_start_index', 'frame_end_index']]

        # store them in the dataframe
        adf = pd.concat([adf, pd.DataFrame({
            "state_1": [state_model[0]],
            "state_2": [state_model[1]],
            "state_3": [state_model[2]],
            "state_4": [state_model[3]],
            "most_likely_state": [['state_1', 'state_2', 'state_3', 'state_4'][np.argmax(state_model)]],
            **count_dict,
            **rest.to_dict()
        })])


In [51]:
adf[[ 'session', 'signed_contrast', 'choice', 'state_1', 'state_2', 'state_3', 'state_4', 'most_likely_state']].head(20)

# adf.to_pickle("./data/adf.pkl")

In [32]:
one_ = ONE(silent=True, password='international')

In [ ]:
# !pip install ibllib
from one.api import ONE
import ibllib.io.video as vidio

In [48]:
val_eid[0]

'db4df448-e449-4a6f-a0e7-288711e7a75a'

In [47]:
# val_eid

url = vidio.url_from_eid(val_eid[0], one=one)['body']
url

AttributeError: module 'one' has no attribute 'eid2path'

In [16]:
cam = one.load_object(val_eid[0], 'leftCamera', collection='alf')
cam.keys()

C:\Users\our\Downloads\ONE\openalyx.internationalbrainlab.org\danlab\Subjects\DY_009\2020-02-27\001\alf\_ibl_leftCamera.dlc.pqt: 100%|██████████| 33.6M/33.6M [00:07<00:00, 4.63MB/s]
C:\Users\our\Downloads\ONE\openalyx.internationalbrainlab.org\danlab\Subjects\DY_009\2020-02-27\001\alf\_ibl_leftCamera.features.pqt: 100%|██████████| 3.46M/3.46M [00:01<00:00, 2.07MB/s]
C:\Users\our\Downloads\ONE\openalyx.internationalbrainlab.org\danlab\Subjects\DY_009\2020-02-27\001\alf\#2024-06-22#\_ibl_leftCamera.lightningPose.pqt: 100%|██████████| 24.4M/24.4M [00:03<00:00, 7.60MB/s]
C:\Users\our\Downloads\ONE\openalyx.internationalbrainlab.org\danlab\Subjects\DY_009\2020-02-27\001\alf\#2023-04-20#\_ibl_leftCamera.times.npy: 100%|██████████| 1.66M/1.66M [00:00<00:00, 2.09MB/s]
C:\Users\our\Downloads\ONE\openalyx.internationalbrainlab.org\danlab\Subjects\DY_009\2020-02-27\001\alf\leftCamera.ROIMotionEnergy.npy: 100%|██████████| 1.66M/1.66M [00:00<00:00, 2.23MB/s]


dict_keys(['dlc', 'features', 'lightningPose', 'times', 'ROIMotionEnergy'])

In [49]:
import os
import numpy as np
from one.api import ONE

# Initialize ONE instance
one = ONE(silent=True, password='international')

# Define your session ID and video dataset name
session_id = 'db4df448-e449-4a6f-a0e7-288711e7a75a'  # Replace with your session ID or other identifiers
# video_name = 'your_video_name'  # Replace with the actual video dataset name
# save_path = 'path/to/save/video.mp4'  # Replace with your desired save path

# Search for a session
session = one.search(eid=session_id)  # Ensure this method is appropriate for your search criteria
session

# Load the video dataset
# video_data = one.load_dataset(dataset_type='video', dataset_name=video_name)  # Adjust parameters based on dataset type and name

# If the video data is a numpy array or similar, you might need to save it properly
# Here's a generic way to handle video saving if you have raw data
# with open(save_path, 'wb') as f:
#     f.write(video_data)

# print(f'Video saved to {save_path}')


ValueError: Invalid search term "eid", see `one.search_terms()`

In [50]:
one.search_terms()

('dataset',
 'date_range',
 'laboratory',
 'number',
 'projects',
 'subject',
 'task_protocol',
 'dataset_qc_lte')